In [1]:
import pandas as pd
import os
import csv

In [2]:
def NormalizeStat(stat, min_, max_):
    return (stat - min_) / (max_ - min_)

In [4]:
df_game_log = pd.read_csv(os.path.join('..', "Resources", "game_logs.csv"))
df_players = pd.read_csv(os.path.join('..', "Resources", "players.csv"))

df_players = df_players.rename(columns={'id':'player_id'})

df_player_game_logs = df_game_log.merge(df_players, how='inner', on='player_id')

df_player_game_logs = df_player_game_logs.set_index('year')

df_player_game_logs.head()

,player_id,date,week,team_x,location,opponent,result,started,rushing_attempts,rushing_yards,...,receptions,receiving_yards,receiving_touchdowns,conversions,fumbles_lost,fumble_touchdowns,name,team_y,team_abbr,position
year,,,,,,,,,,,,,,,,,,,,,
2017,KamaAl00,2017-09-11,1,NOR,Away,MIN,L,False,7,18,...,4,20,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-09-17,2,NOR,Home,NWE,L,False,1,3,...,3,51,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-09-24,3,NOR,Away,CAR,W,False,2,37,...,3,5,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-10-01,4,NOR,Away,MIA,W,False,5,25,...,10,71,1,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB
2017,KamaAl00,2017-10-15,6,NOR,Home,DET,W,True,10,75,...,3,12,0,0,0,0,Alvin Kamara,New Orleans Saints,NOR,RB


In [5]:
df_2020_running_backs = df_player_game_logs.loc[df_player_game_logs.index.isin([2020]) & (df_player_game_logs.position == 'RB')]

df_2020_running_backs['rushing_yard_rank'] = [NormalizeStat(r, min(df_2020_running_backs['rushing_yards']), max(df_2020_running_backs['rushing_yards'])) for r in df_2020_running_backs['rushing_yards']]
df_2020_running_backs['rushing_td_rank'] = [NormalizeStat(r, min(df_2020_running_backs['rushing_touchdowns']), max(df_2020_running_backs['rushing_touchdowns'])) for r in df_2020_running_backs['rushing_touchdowns']]

df_2020_running_backs.eval("overall_ranking = rushing_yard_rank + rushing_td_rank", inplace=True)

df_grouped_game_log = df_2020_running_backs.groupby(['name', 'year'])

print(df_grouped_game_log['rushing_yards'].sum().nlargest(5),
df_grouped_game_log['overall_ranking'].sum().nlargest(10))

name           year
Dalvin Cook    2020    489
Josh Jacobs    2020    377
Derrick Henry  2020    376
Todd Gurley    2020    375
Aaron Jones    2020    374
Name: rushing_yards, dtype: int64 name             year
Dalvin Cook      2020    5.177893
Josh Jacobs      2020    3.930915
Todd Gurley      2020    3.920553
Ezekiel Elliott  2020    3.863558
Derrick Henry    2020    3.530225
Aaron Jones      2020    3.519862
Nick Chubb       2020    3.317789
Kareem Hunt      2020    3.108808
Alvin Kamara     2020    3.100173
James Robinson   2020    3.036269
Name: overall_ranking, dtype: float64


In [6]:
df_2020_pit_wide_receivers = df_player_game_logs.loc[df_player_game_logs.index.isin([2020]) & (df_player_game_logs.position == 'WR') & (df_player_game_logs.team_abbr == 'PIT')]

df_2020_pit_wide_receivers['receiving_touchdown_rank'] = [NormalizeStat(r, min(df_2020_pit_wide_receivers['receiving_touchdowns']), max(df_2020_pit_wide_receivers['receiving_touchdowns'])) for r in df_2020_pit_wide_receivers['receiving_touchdowns']]

df_2020_pit_wide_receivers_grouped = df_2020_pit_wide_receivers.groupby(['name'])

print(df_2020_pit_wide_receivers_grouped['receiving_touchdown_rank'].sum().nlargest(10))

name
Chase Claypool         1.333333
JuJu Smith-Schuster    1.000000
Diontae Johnson        0.333333
James Washington       0.333333
Ray-Ray McCloud        0.000000
Name: receiving_touchdown_rank, dtype: float64


In [7]:
df_2020_game_logs = df_player_game_logs.loc[df_player_game_logs.index.isin([2020])]
print(df_2020_game_logs.columns)

df_2020_groupby_opponent = df_2020_game_logs.groupby('opponent')

df_2020_groupby_opponent[['rushing_attempts', 'rushing_yards', 'rushing_touchdowns']].sum().sort_values(by='rushing_touchdowns', ascending=False)

Index(['player_id', 'date', 'week', 'team_x', 'location', 'opponent', 'result',
       'started', 'rushing_attempts', 'rushing_yards', 'rushing_touchdowns',
       'pass_targets', 'receptions', 'receiving_yards', 'receiving_touchdowns',
       'conversions', 'fumbles_lost', 'fumble_touchdowns', 'name', 'team_y',
       'team_abbr', 'position'],
      dtype='object')


,rushing_attempts,rushing_yards,rushing_touchdowns
opponent,,,
LVR,131,634,10
NYJ,154,643,9
WAS,153,648,8
MIA,127,630,8
PHI,145,571,8
CAR,123,667,8
DAL,167,779,7
BUF,126,543,7
NYG,149,553,6


In [81]:
schedule = {
    'home': ['MIN', 'TAM', 'CAR', 'NWE', 'TEN', 'PHI', 'JAX', 'MIA', 'PIT', 'IND', 'NYG', 'SFO'],
    'away': ['ATL', 'GNB', 'CHI', 'DEN', 'HOU', 'BAL', 'DET', 'NYJ', 'CLE', 'CIN', 'WAS', 'LAR']
}

schedule = pd.DataFrame(schedule)

df_team_logs = df_2020_game_logs.loc[:, ['team_x', 'opponent', 'date', 'rushing_attempts', 'rushing_yards', 'rushing_touchdowns', 'receptions', 'receiving_yards', 'receiving_touchdowns']]

df_team_logs = df_team_logs.rename(columns={"team_x":'team'})

df_2020_groupby_team = df_team_logs.groupby(['team', 'date'])
df_2020_groupby_opp = df_team_logs.groupby(['opponent', 'date'])


df_2020_stats_for = df_2020_groupby_team['receiving_yards'].sum() # , 'rushing_attempts', 'rushing_touchdowns', 'receptions', 'receiving_yards', 'receiving_touchdowns']].sum()
df_2020_stats_against = df_2020_groupby_opp['receiving_yards'].sum() # , 'rushing_attempts', 'rushing_touchdowns', 'receptions', 'receiving_yards', 'receiving_touchdowns']].sum()

df_1 = df_2020_stats_for.groupby('team').mean().sort_values(ascending=False).reset_index()
df_2 = df_2020_stats_against.groupby('opponent').mean().sort_values(ascending=False).reset_index()

df_average_rushing_yards = pd.merge(df_1, df_2, left_on='team', right_on='opponent', suffixes=('_for', '_against'))

df_average_rushing_yards.drop(columns='opponent', inplace=True)

# df_average_rushing_yards.head()

df_3 = schedule.merge(df_average_rushing_yards, left_on='home', right_on='team').merge(df_average_rushing_yards, left_on='away', right_on='team', suffixes=('_home', '_away')).head()

df_3.eval('receiving_value_home = receiving_yards_for_home + receiving_yards_against_away', inplace=True)

df_3.nlargest(columns={'receiving_value_home'}, n=5)



,home,away,team_home,receiving_yards_for_home,receiving_yards_against_home,team_away,receiving_yards_for_away,receiving_yards_against_away,receiving_value_home
0,MIN,ATL,MIN,226.40,283.20,ATL,294.40,345.60,572.00
1,TAM,GNB,TAM,275.00,265.40,GNB,303.50,269.00,544.00
2,CAR,CHI,CAR,292.00,229.00,CHI,247.20,243.80,535.80
3,NWE,DEN,NWE,226.00,244.00,DEN,240.75,275.75,501.75
4,TEN,HOU,TEN,249.25,272.75,HOU,290.20,242.60,491.85
